<a href="https://colab.research.google.com/github/UOS-COMP1201/code/blob/main/integerprogramming.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
## Python implementation of questions 5&6 in tutorial 11
## follow the "open in Colab" link to run the code in Google Colab
import numpy as np
from scipy.optimize import linprog
import copy

In [2]:
def IP(c,A,b,bounds,depth):
  global min_val,min_vars,DEBUG
  res=linprog(c,A,b,bounds=bounds)
  if res.fun == None:
    if DEBUG:
      print("pruned by infeasibility, depth {}".format(depth))
    return 
  if res.fun>min_val:
    if DEBUG:
      print(res.fun,res.x)
      print("pruned by bound, depth {}".format(depth))
    return 
  result=res.fun
  # at this point result<min_val
  if DEBUG: print(result,res.x)
  int_sol=True
  for var in res.x:
    if np.floor(var)!=np.ceil(var):
      int_sol=False
      break
  if  int_sol:
        min_val=result
        min_vars=res.x
        return
  # not all variables are integers
  # if a variable is not int branch on ceil and floor      
  for i,var in enumerate(res.x):
    fl=np.floor(var)
    cl=np.ceil(var)
    
    if fl!=cl:
      if DEBUG: print("branching on {} {} , depth {}".format(fl,cl,depth))
      # bounds is an array of pairs
      tmp=copy.deepcopy(bounds)
      # include the ceiling constraint and recurse
      tmp[i]=(cl,tmp[i][1])
      IP(c,A,b,tmp,depth+1)
      # include the floor constraint and recurse
      tmp=copy.deepcopy(bounds)
      tmp[i]=(tmp[i][0],fl)
      IP(c,A,b,tmp,depth+1)
   
  
  #return res.fun,res.x

In [3]:

c=np.array([4,5])
A=np.array([[1,4],[3,-4]])  
b=np.array([101,70])
# the solver minimises the objective function
# so we need to negate the objective function to maximise it
c=[-x for x in c]

bounds=[(0,None) for i in range(len(c))]    
min_val=np.inf
min_vars=None
DEBUG=False

r=IP(c,A,b,bounds,0)


In [4]:
print(-min_val)
print(min_vars)

239.0
[41. 15.]


### Vertex cover

In [5]:
c=np.array([2,2,7,2,2.])

b=np.array([-1,-1,-1,-1,-1,-1])
A=np.array([[-1,-1,0,0,0],[-1,0,-1,0,0],[-1,0,0,-1,0],[0,-1,-1,0,0],[0,0,-1,-1,0],[0,0,-1,0,-1]  ])
bounds=[(0,None) for i in range(len(c))]    

In [6]:
min_val=np.inf
min_vars=None
DEBUG=False
r=IP(c,A,b,bounds,0)

In [7]:
print(min_val)
print(min_vars)

8.0
[1. 1. 0. 1. 1.]
